<a href="https://colab.research.google.com/github/neko88/GoogleCollab/blob/main/Template_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import cv2
import imghdr
import tensorflow as tf
from tensorflow import keras
import os
from matplotlib import pyplot as plt
import numpy as np

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
data_dir = 'root_data_folder'

In [4]:
image_exts = ['jpeg','jpg','bmp','png']

In [ ]:
img = cv2.imread(os.path.join('root_data_folder','data_folder','filename_ext'))

In [ ]:
img.shape   # (h,w,colour channel)

In [ ]:
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:
# Clear garbage data
for image_class in os.listdir(data_dir):
  for image in os.listdir(os.path.join(data_dir, image_class)):
    image_path = os.path.join(data_dir, image_class, image)
    try:
      img = cv2.imread(image_path)      # Check that it can be opened by OpenCV
      tip = imghdr.what(image_path)     # Check that it is an acceptable extension
      if tip not in image_exts:
        print ('Image not in ext list {}'.format(image_path))
        os.remove(image_path)
    except Exception as e:
      print('Issue with image {}'.format(image_path))
      os.remove(image_path)

In [ ]:
# LOAD THE DATA - build the data pipeline
data = tf.keras.utils.image_dataset_from_directory('root_data_folder')



In [ ]:
# Iterator to load into memory - allows access data pipeline
data_iterator = data.as_numpy_iterator()      

In [ ]:
# Access the data pipeline
# batch[i] : lists the img's classes contained in batch i
# batch[i].shape : (batch_size, h, w, colour)
batch = data_iterator.next()

In [ ]:
# PREPROCESS & STANDARDIZE THE DATA
data = data.map(lambda x, y: (x/255, y))

In [ ]:
scaled_iterator = data.as_numpy_iterator()
# scaled_iterator.next()[i]

In [ ]:
batch = scaled_iterator.next()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
  ax[idx].imshow(img)
  ax[idx].title.set_text(batch[1][idx])

In [ ]:
# SPLIT THE DATA 
# size is the number of batches
train_size = int(len(data)*.7)
val_size = int(len(data)*.2) + 1
test_size = int(len(data)*.1) + 1


In [ ]:
# Allocate data based on our desired batch sizes
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [ ]:
### DEEP MODEL, adding layers
# alt: model = Sequential()
# alt: model.add(Conv2D...)

model = tf.keras.Sequential([
    # Adding first layer as the input layer with 16 filters, filter size 3x3, stride 1 / 'relu' - output converts negative to 0, and keeps positives /
    tf.keras.layers.Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)),
    # Scans 2x2 regions and outputs max value
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(32, (3,3), 1, activation='relu'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(16, (3,3), 1, activation='relu'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# Compile the model
#alt: model.compile('adam, loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    metrics = ['accuracy']
)

In [ ]:
# View the model
# alt: model.summary()


In [ ]:
### TRAIN THE MODEL
# We can log the training of our model:
logdir = 'log_directory'
tensorboard_callback = tf.keras.callbackTensorBoard(log_dir=logdir)

In [ ]:
# Train the model
hist = model.fit(train, epochs=20, validation_data = val, callbacks=[tensorboard_callback])


In [ ]:
# Lists all the loss and accuracy results
hist.history

In [ ]:
### VISUALIZE THE MODEL
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc='upper left')
plt.show()

In [ ]:
## EVALUATE PERFORMANCE
# from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
pre = tf.keras.metrics.Precision
rec = tf.keras.metrics.Recall
acc = tf.keras.metrics.BinaryAccuracy

In [ ]:
# Test out our model
for batch in test.as_numpy_iterator():
  X, y = batch
  yhat = model.predict(X)
  pre.update_status(y, yhat)
  rec.update_status(y, yhat)
  acc.update_status(y, yhat)

In [ ]:
print(f'Precision:{pre.result().numpy()}, Recall:{rec.result().numpy()}, Accuracy:{acc.result().numpy()}')

In [ ]:
### TEST THE DATA
img_test = cv2.imread('file_name')
resize = tf.image.resize(img, (256,256))
plt.imshow(cv2.cvtColor(resize.numpy().astype(int), cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
# Rewraps in another array
# divide by 255 to scale
yhat = model.predict(np.expand_dims(resize/255,0))
# The value will be a value between 0(A) to 1(B)

In [ ]:
if yhat > 0.5:
  print(f'Predicted class is B')
else:
  print(f'Predicted class is A')

In [ ]:
#### SAVE THE MODEL
# from tensorflow.keras.models import load_model

In [ ]:
model.save(os.path.join('folder_name','my_model_name.h5'))

In [ ]:
new_model = load_model(os.path.join('folder_name','my_model_name.h5'))